In [18]:
import torch

from collections import defaultdict
from importlib import import_module
from torch.utils.data import DataLoader
from tqdm import tqdm

from skp.toolbox.functions import load_model_from_config

In [2]:
cfg_name = "mammo.cfg_ddsm_pretrained_aux_losses"
cfg = import_module(f"skp.configs.{cfg_name}").cfg
weights_path = cfg.save_dir + cfg_name + "/ce2bf46d/fold0/checkpoints/ema_weights.pt"
model = load_model_from_config(cfg, weights_path, device="cuda:0", eval_mode=True)
dataset = import_module(f"skp.datasets.{cfg.dataset}").Dataset(cfg, mode="val")

/home/ian/miniconda3/envs/skp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ian/miniconda3/envs/skp/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Enabling gradient checkpointing ...
Loading pretrained backbone from /home/ian/projects/SKP/experiments/mammo/mammo.cfg_ddsm_seg_cls_v02/445e5ce2/fold0/checkpoints/last.ckpt ...
missing keys: ['conv_head.weight', 'bn2.weight', 'bn2.bias', 'bn2.running_mean', 'bn2.running_var']
Loading weights from /home/ian/projects/SKP/experiments/mammo/mammo.cfg_ddsm_pretrained_aux_losses/ce2bf46d/fold0/checkpoints/ema_weights.pt ...


In [19]:
loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=13, drop_last=False)

In [20]:
pred_dict = defaultdict(list)
for batch in tqdm(loader, total=len(loader)):
    x = batch["x"]
    x = x.to("cuda:0")
    with torch.inference_mode():
        p = model({"x": x})
    pred_dict["y_pred"].append(p["logits"][:, 0].cpu().numpy())
    pred_dict["y_true"].append(batch["y"][:, 0].numpy())
    pred_dict["group_index"].append(batch["group_index"].numpy())

  0%|          | 0/684 [00:00<?, ?it/s]/home/ian/miniconda3/envs/skp/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/ian/miniconda3/envs/skp/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
 30%|██▉       | 203/684 [01:30<03:33,  2.26it/s]

defaultdict(list,
            {'y_pred': [array([-6.0096784, -5.664594 , -6.4485154, -4.9985366, -7.8987017,
                     -5.416729 , -5.286799 , -7.283538 , -6.1503277, -6.6345015,
                     -5.3408723, -6.632629 , -8.614459 , -4.2328224, -6.6327295,
                     -7.6492496], dtype=float32)],
             'y_true': [array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                    dtype=float32)],
             'group_index': [array([ 4,  4,  5,  5, 18, 18, 19, 19, 26, 26, 27, 27, 36, 36, 36, 37])]})